In [1]:
#!pip3 install beautifulsoup4

In [2]:
#!pip3 install lxml

In [2]:
#Представим, что перед нами стоит задача проанализировать, какие слова необходимо учить человеку,
#недостаточно владеющему техническим английским, для успешной работы в области Data Science
#То есть нам нужно получить слова, наиболее часто встречающиеся в основной технической документации DS

#Для решения задачи мы спарсим весь текст с сайтов документации sympy, pandas, skilearn
#используя рекурсивный обход страниц сайта "в глубину" (как графа)
#текст разберём на слова, приведём их к нормальной форме и составим статистики

In [99]:
import requests
from bs4 import BeautifulSoup

In [100]:
### Общие настройки парсинга (глубина парсинга и User-agent)
depth = 3
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'}

In [101]:
### Функции ###

In [102]:
def page_directory(page):
    '''Фунция, которая по строке адреса страницы даёт строку адреса её директории'''
    return page.url.rpartition('/')[0] + '/'

In [103]:
def take_all_text_and_links_from_page(link):
    '''Функция выбора с текущей веб-страницы всего текста и всех ссылок, ведущих не более чем на depth уровней выше'''
    page = requests.get(link, headers = headers)
    soup = BeautifulSoup(page.content, "lxml")
    #Получим весь текст
    p_tags = soup.find_all('p')
    text =  [p_tag.text for p_tag in p_tags]
    #И все ссылки
    links = soup.find_all('a')
    links = [link.get('href').rpartition('.html')[0] + '.html' for link in links]
    #Удалим все дубликаты и ссылки, начинающиеся с точки
    links = list(set(links))
    links = list(filter( lambda x : not x.startswith("."), links ))
    #Возможно для типа ссылок: с абсолютными путями и оносительными. Приведём все ссылки к абсолютному виду
    page_directory_link = page_directory(page)
    links = [page_directory_link + current_link if not current_link.startswith("http") else current_link for current_link in links]
    #Ну и, наконец, оставим только ссылки, ведущие не более чем на depth уровней выше от исходной
    links = list(filter( lambda x : x.startswith(page_directory_link), links ))
    links = list(filter( lambda x : (x.count('/') - int_basic_level) < depth, links ))
    return text, links

In [104]:
def graph_recursion(link, all_text, all_links):
    '''Функция рекурсивного обхода страниц сайта как графа в глубину, в которую как начальную вершину мы передадим базовую страницу'''
    #В множество посещённых страничек добавим ту, что посещаем в данный момент
    all_links.append(link)
    #В весь текст с текущего домена добавим текст полученный с текущей ссылки
    new_text, new_links = take_all_text_and_links_from_page(link)
    all_text += new_text
    #А к тем ссылкам с текущей страницы, которые ещё не были посещены, рекурсивно применим эту же функцию
    new_links = list(set(new_links) - set(all_links))
    for new_link in new_links:
        graph_recursion(new_link, all_text, all_links)

In [105]:
###Сайты c документацией по основным библиотекам Data Science ###
doc_urls = ['https://docs.sympy.org/latest/index.html',
            'https://pandas.pydata.org/docs/user_guide/index.html',
            'https://scikit-learn.org/dev/user_guide.html']

In [106]:
#Для сокращения времени выполнения проверим работоспособность на сайте scikit-learn
#Далее при потребность уже легко обернуть в цикл по всему массиву ссылок
basic_link = doc_urls[2]

##Начальный уровень парсинга
int_basic_level = basic_link.count('/')

In [107]:
#Непосредственно сам процесс парсинга (получение всего содержимого сайта со всех страниц)
all_text_from_domen = []
all_links_from_domen = []
graph_recursion(basic_link, all_text_from_domen, all_links_from_domen)

In [108]:
### Обработка полученного текста ###
import re
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [109]:
words_in_string = []

for raw_text in all_text_from_domen:
    # формируем датасет из отдельных слов, приводим к нижнему регистру
    raw_text_lower = raw_text.lower()
    # разбиваем текст на слова
    regular_expr = r'\w+'
    reg_expr_compiled = re.compile(regular_expr)
    text_by_words = reg_expr_compiled.findall(raw_text_lower) 
    text_by_words = list(filter(lambda x: x.isalpha(), text_by_words))
    #Cразу при добавлении приводим слова к "стандартной" форме ("счастливы" -> "счастье")
    for word in text_by_words:
        words_in_string.append(morph.parse(word)[0].normal_form)  

In [110]:
#Собираем статистики по словам
import pandas as pd

tokens_df = pd.DataFrame({'word': words_in_string})
# дамми-столбец
tokens_df = tokens_df.assign(dummy = 1)
# аггрегируем статистики
word_count_df = tokens_df.groupby(['word'])['dummy'].count().reset_index().sort_values(by='dummy', ascending=False)

In [111]:
#Выведем первые 10 строк
word_count_df.rename(columns = {'dummy' : 'count'}, inplace=True)
word_count_df.head(10)

,word,count
11355,the,108833
7874,of,49870
11504,to,38485
0,a,32107
5561,is,30707
441,and,27269
5204,in,26432
4109,for,24904
969,be,14236
5103,if,13371


In [112]:
#Ожидаемо, получили союзы. Отфильтруем датафрейм по длине слова
word_count_df = word_count_df[word_count_df['word'].str.len() > 3]
word_count_df.head(10)

,word,count
11404,this,12860
12462,with,11677
11354,that,11241
2557,data,10600
3581,estimator,8205
7801,number,7632
3266,each,6886
12428,will,6691
9942,samples,6497
8216,parameters,6142


In [114]:
#Уже лучше. Сохраним результаты в файл, чтобы в следующий раз уже просто считать его, выполнив ячейку ниже:
import os.path

In [118]:
if os.path.exists('english_words.csv'):
    #Если файл уже существует - считываем его
    word_count_df  = pd.read_csv('english_words.csv', index_col='num')
else:
    #При первом же запуске нам нужно создать соответствующий CSV файл
    word_count_df.index.names = ['num']
    word_count_df.to_csv('english_words.csv')

In [120]:
#Получили желаемое: сортированный по частоте список слов в нормальной форме, встречающихся на всех страницах сайта
#и количество их вхождений. 

#В дальнейшем данный файл будет расширен получением столбца с переводом слов и 
#автоматизацией логики изучения по алгоритму интервального повторения, с сохранением и визуализацией прогресса